-------------------

#### desde aqui

In [3]:
import pandas as pd  # Importa la librería pandas para el manejo de datos
from sklearn.model_selection import train_test_split  # Importa la función train_test_split de sklearn para dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.ensemble import RandomForestRegressor  # Importa el modelo RandomForestRegressor de sklearn
from sklearn.metrics import r2_score

# Lee el archivo de datos de los taxis para el año 2023 en formato parquet y lo guarda en el DataFrame taxis_NYC
taxis_NYC =pd.read_parquet('../dataset limpios api/taxis_2023.parquet')

# Filtra los viajes con distancia menor o igual a 100 millas
taxis_NYC = taxis_NYC[taxis_NYC['trip_distance'] <= 100]

# Filtra los viajes con costo mayor o igual a 5 dólares
taxis_NYC = taxis_NYC[taxis_NYC['amount'] >= 5] 

# Elimina los viajes que tienen como punto de recogida el aeropuerto Newark
taxis_NYC = taxis_NYC[taxis_NYC['pickup_borough'] != 'EWR']

# Extrae el año, mes, día y día de la semana de la fecha de recogida
taxis_NYC['año'] = pd.to_datetime(taxis_NYC['pickup_date']).dt.year
taxis_NYC['mes'] = pd.to_datetime(taxis_NYC['pickup_date']).dt.month
taxis_NYC['dia'] = pd.to_datetime(taxis_NYC['pickup_date']).dt.day
taxis_NYC['dayofweek'] = pd.to_datetime(taxis_NYC['pickup_date']).dt.dayofweek

# Utiliza la hora de recogida como hora de viaje
taxis_NYC['hora'] = taxis_NYC['pickup_hour']

# Rellena las horas con un 0 al principio y agrega ':00' para formatearlas como HH:00
taxis_NYC['hora'] = taxis_NYC['hora'].str.zfill(2) + ':00'

# Agrupa los viajes por tipo de servicio, punto de recogida, día de la semana y hora, y cuenta la cantidad de viajes
taxis_NYC = taxis_NYC.groupby(['service_type', 'pickup_borough', 'año', 'mes', 'dia', 'dayofweek', 'hora']).agg({'passenger_count': 'count'}).reset_index()

# Renombra la columna resultante como 'demand'
taxis_NYC.columns = ['service_type', 'pickup_borough', 'año', 'mes', 'dia', 'dayofweek', 'hora', 'demand']

# Agrupa los viajes por tipo de servicio, punto de recogida, día de la semana y hora, y cuenta la cantidad de viajes (de nuevo)
taxis_NYC = taxis_NYC.groupby(['service_type', 'pickup_borough', 'dayofweek','hora']).size().reset_index(name='demand')

# Convierte el DataFrame resultante en un DataFrame de pandas
taxis_NYC = pd.DataFrame(taxis_NYC)

# Normaliza la columna 'demand' dividiendo cada valor por el máximo valor y multiplicándolo por 100
taxis_NYC['demand'] = taxis_NYC['demand'].apply(lambda x :  (x / taxis_NYC['demand'].max()))
taxis_NYC['demand'] = round(taxis_NYC['demand']*100,2)

# Asigna un ID numérico a los tipos de servicio ('yellow': 1, 'green': 0)
taxis_NYC['serviceID'] = taxis_NYC['service_type'].map({'yellow': 1, 'green': 0})

# Asigna un ID numérico a los puntos de recogida ('Manhattan': 1, 'Queens': 2, 'Brooklyn': 3, 'Bronx': 4, 'Staten Island': 5)
taxis_NYC['borough2ID'] = taxis_NYC['pickup_borough'].map({'Manhattan': 1, 'Queens': 2, 'Brooklyn': 3, 'Bronx': 4, 'Staten Island': 5})

# Elimina las columnas 'service_type' y 'pickup_borough' del DataFrame
taxis_NYC = taxis_NYC.drop(columns={'service_type','pickup_borough'})

# Convierte la hora de formato HH:MM a un valor numérico de 0 a 23
taxis_NYC['hora'] = pd.to_datetime(taxis_NYC['hora'], format='%H:%M').dt.hour

# Asigna el DataFrame resultante a la variable df
df = taxis_NYC

# Separa las características (X) y la variable objetivo (y)
X = df.drop(['demand'], axis=1)
y = df['demand']

# Divide los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=65)

# Inicializa el modelo RandomForestRegressor con una semilla aleatoria
model_taxi = RandomForestRegressor(random_state=10)

# Entrena el modelo con los datos de entrenamiento
model_taxi.fit(X_train, y_train)

#
#
#-------- desde aqui pueden omitir ya que hemos dicho q el modelo lo subiriamos del local o si desean y tienen tiempo y si es mejor si lo pponen pero eso tiene q guardarse en un backet

# Evaluación de modelos
y_pred = model_taxi.predict(X_test) # por si quieres testear y hacer graficas

r2_taxi = r2_score(y_test, y_pred)

# Guarda el modelo entrenado en un archivo llamado 'model_taxi.pkl'
import joblib
joblib.dump(model_taxi, 'model_taxi.pkl') # por si crean un baket y lo guardan


['model_taxi2.pkl']

### Hasta aqui

--------------------

esto es para validar pero va en el seikmeiker

In [4]:
# Suponiendo que tienes algunos datos para hacer predicciones
nuevos_datos = {
    'dia': [1, 2, 4],  # Ejemplo de valores de día
    'hora': ['17:00', '2:00', '21:00'],  # Ejemplo de valores de hora
    'service_type2': [1, 0, 0],  # Ejemplo de valores de service_type2
    'pickup_borough2': ['1', '2', '5']  # Ejemplo de valores de pickup_borough2
} #dia	hora	total_millas_recorridas	ganancia_hora	service_type2	pickup_borough2
# Crear DataFrame de nuevos datos
X_nuevos_datos = pd.DataFrame(nuevos_datos)
X_nuevos_datos['hora'] = pd.to_datetime(X_nuevos_datos['hora'], format='%H:%M').dt.hour

# Cargar el modelo desde el archivo pickle
loaded_model = joblib.load('model_taxi2.pkl')

# Suponiendo que tienes nuevos datos en X_nuevos_datos
# Hacer predicciones con el modelo cargado
y_pred_ml = loaded_model.predict(X_nuevos_datos)

# Imprimir las predicciones
print("Predicciones del modelo cargado:", y_pred_ml)

Predicciones del modelo cargado: [98.3316 64.6907  1.8927]


c:\Users\Pc\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- dia
- pickup_borough2
- service_type2
Feature names seen at fit time, yet now missing:
- borough2ID
- dayofweek
- serviceID

  warnings.warn(message, FutureWarning)
